# Imports

In [14]:
!pip3 install hazm
!apt-get install antiword
!pip3 install textract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
antiword is already the newest version (0.37-11build1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [15]:
import textract
import hazm
import re
import pandas as pd

from keras.layers import Concatenate 
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

# Preprocessing

In [16]:
full_text = textract.process("/content/daftar2.doc")
full_text = full_text.decode("utf-8") 
full_text = re.sub(r'\u200f', ' ', full_text)
full_text = re.sub(r'\n', '.', full_text)
full_text = re.sub(r'\s+', ' ', full_text)

full_text

'. مدتى اين مثنوى تاخير شد مهلتى بايست تا خون شير شد. تا نزايد بخت تو فرزند نو خون، نگردد شير شيرين خوش شنو. چون ضياء الحق حسام الدين عنان باز گردانيد ز اوج آسمان . چون به معراج حقايق رفته بود بى بهارش غنچه ها نشكفته بود. چون ز دريا سوى ساحل باز گشت چنگ شعر مثنوى با ساز گشت . مثنوى كه صيقل ارواح بود باز گشتش روز استفتاح بود. مطلع تاريخ اين سودا و سود سال اندر ششصد و شصت و دو بود. بلبلى ز ينجا برفت و باز گشت بهر صيد اين معانى باز گشت . ساعد شه مسكن اين باز باد تا ابد بر خلق اين در باز باد. آفت اين در هوا و شهوت است ور نه اينجا شربت اندر شربت است . اين دهان بر بند تا بينى عيان چشم بند آن جهان حلق و دهان . اى دهان تو خود دهانه ى دوزخى وى جهان تو بر مثال برزخى . نور باقى پهلوى دنياى دون شير صافى پهلوى جوهاى خون . چون در او گامى زنى بى احتياط شير تو خون مى شود از اختلاط. يك قدم زد آدم اندر ذوق نفس شد فراق صدر جنت طوق نفس . همچو ديو از وى فرشته مى گريخت بهر نانى چند آب چشم ريخت . گر چه يك مو بد گنه كاو جسته بود ليك آن مو در دو ديده رسته بود. بود آدم ديده ى نور قديم موى در ديده بود كوه عظيم .

In [17]:
full_text_list = full_text.split('.')
for index, item in enumerate(full_text_list):
  if item == '':
    full_text_list.remove('')
  else:
    full_text_list[index] = full_text_list[index].lstrip().rstrip()
full_text_list

[' مدتى اين مثنوى تاخير شد مهلتى بايست تا خون شير شد',
 'تا نزايد بخت تو فرزند نو خون، نگردد شير شيرين خوش شنو',
 'چون ضياء الحق حسام الدين عنان باز گردانيد ز اوج آسمان',
 'چون به معراج حقايق رفته بود بى بهارش غنچه ها نشكفته بود',
 'چون ز دريا سوى ساحل باز گشت چنگ شعر مثنوى با ساز گشت',
 'مثنوى كه صيقل ارواح بود باز گشتش روز استفتاح بود',
 'مطلع تاريخ اين سودا و سود سال اندر ششصد و شصت و دو بود',
 'بلبلى ز ينجا برفت و باز گشت بهر صيد اين معانى باز گشت',
 'ساعد شه مسكن اين باز باد تا ابد بر خلق اين در باز باد',
 'آفت اين در هوا و شهوت است ور نه اينجا شربت اندر شربت است',
 'اين دهان بر بند تا بينى عيان چشم بند آن جهان حلق و دهان',
 'اى دهان تو خود دهانه ى دوزخى وى جهان تو بر مثال برزخى',
 'نور باقى پهلوى دنياى دون شير صافى پهلوى جوهاى خون',
 'چون در او گامى زنى بى احتياط شير تو خون مى شود از اختلاط',
 'يك قدم زد آدم اندر ذوق نفس شد فراق صدر جنت طوق نفس',
 'همچو ديو از وى فرشته مى گريخت بهر نانى چند آب چشم ريخت',
 'گر چه يك مو بد گنه كاو جسته بود ليك آن مو در دو ديده رسته بود',
 'بود آدم 

In [18]:
word_tokenized =  [ hazm.word_tokenize(text) for text in full_text_list  ]
word_tokenized

# chasboondan
# TOKENIZER KERAS 

[['مدتى',
  'اين',
  'مثنوى',
  'تاخير',
  'شد',
  'مهلتى',
  'بايست',
  'تا',
  'خون',
  'شير',
  'شد'],
 ['تا',
  'نزايد',
  'بخت',
  'تو',
  'فرزند',
  'نو',
  'خون',
  '،',
  'نگردد',
  'شير',
  'شيرين',
  'خوش',
  'شنو'],
 ['چون',
  'ضياء',
  'الحق',
  'حسام',
  'الدين',
  'عنان',
  'باز',
  'گردانيد',
  'ز',
  'اوج',
  'آسمان'],
 ['چون',
  'به',
  'معراج',
  'حقايق',
  'رفته_بود',
  'بى',
  'بهارش',
  'غنچه',
  'ها',
  'نشكفته',
  'بود'],
 ['چون',
  'ز',
  'دريا',
  'سوى',
  'ساحل',
  'باز',
  'گشت',
  'چنگ',
  'شعر',
  'مثنوى',
  'با',
  'ساز',
  'گشت'],
 ['مثنوى',
  'كه',
  'صيقل',
  'ارواح',
  'بود',
  'باز',
  'گشتش',
  'روز',
  'استفتاح',
  'بود'],
 ['مطلع',
  'تاريخ',
  'اين',
  'سودا',
  'و',
  'سود',
  'سال',
  'اندر',
  'ششصد',
  'و',
  'شصت',
  'و',
  'دو',
  'بود'],
 ['بلبلى',
  'ز',
  'ينجا',
  'برفت',
  'و',
  'باز',
  'گشت',
  'بهر',
  'صيد',
  'اين',
  'معانى',
  'باز',
  'گشت'],
 ['ساعد',
  'شه',
  'مسكن',
  'اين',
  'باز',
  'باد',
  'تا',
  'ابد',
  'بر',
  'خلق

In [25]:
# Persian stop words source: https://github.com/kharazi/persian-stopwords
# stop word
stop_words_df = pd.read_csv("/content/stopwords.txt", sep="\n", header=None, names=["words"])
stop_words_list = stop_words_df["words"].values.tolist()

sentences_list = []
for sentence in word_tokenized:
  print
  words_list = []
  for word in sentence:
    if word not in stop_words_list:
      words_list.append(word)
  sentences_list.append(words_list)
sentences_list

[['مدتى', 'مثنوى', 'تاخير', 'مهلتى', 'بايست', 'خون', 'شير'],
 ['نزايد', 'بخت', 'فرزند', 'نو', 'خون', 'نگردد', 'شير', 'شيرين', 'شنو'],
 ['ضياء', 'الحق', 'حسام', 'الدين', 'عنان', 'گردانيد', 'اوج', 'آسمان'],
 ['معراج', 'حقايق', 'رفته_بود', 'بهارش', 'غنچه', 'نشكفته'],
 ['دريا', 'سوى', 'ساحل', 'چنگ', 'شعر', 'مثنوى', 'ساز'],
 ['مثنوى', 'صيقل', 'ارواح', 'گشتش', 'استفتاح'],
 ['مطلع', 'تاريخ', 'سودا', 'سود', 'سال', 'اندر', 'ششصد', 'شصت'],
 ['بلبلى', 'ينجا', 'برفت', 'صيد', 'معانى'],
 ['ساعد', 'شه', 'مسكن', 'ابد', 'خلق'],
 ['آفت', 'هوا', 'شهوت', 'اينجا', 'شربت', 'اندر', 'شربت'],
 ['دهان', 'بند', 'بينى', 'عيان', 'چشم', 'بند', 'جهان', 'حلق', 'دهان'],
 ['دهان', 'دهانه', 'دوزخى', 'جهان', 'برزخى'],
 ['نور',
  'باقى',
  'پهلوى',
  'دنياى',
  'دون',
  'شير',
  'صافى',
  'پهلوى',
  'جوهاى',
  'خون'],
 ['گامى', 'زنى', 'احتياط', 'شير', 'خون', 'اختلاط'],
 ['قدم',
  'زد',
  'آدم',
  'اندر',
  'ذوق',
  'نفس',
  'فراق',
  'صدر',
  'جنت',
  'طوق',
  'نفس'],
 ['ديو', 'فرشته', 'گريخت', 'نانى', 'آب', 'چشم', 'ريخت'

# Model

In [ ]:
vocab_size = 10000
embed_size = 50
word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1))
word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

model = Sequential()
model.add(Concatenate([word_model, context_model]))
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
model.compile(loss="mean_squared_error", optimizer="rmsprop")



# Result